# DuckDB + Parquet Data Exploration Template

In [ ]:
import duckdb
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from branca.colormap import linear
from pathlib import Path
from shapely.geometry import box
import cartopy.io.shapereader as shpreader
from tqdm import tqdm

from src.util import make_cell_geom, create_config

# --- Configuration ---

In [ ]:
BASE = Path("/Users/kyledorman/data/planet_coverage/dove/results_2023/")  # <-- update this

gpkg_path = Path("/Users/kyledorman/data/planet_coverage/points_in_coastal_strips.gpkg")  # <-- update this

config_file = Path("/Users/kyledorman/data/planet_coverage/dove/results_2023/config.yaml")  # <-- update this
config = create_config(config_file)

# Example path patterns
all_files_pattern = str(BASE / "*/*/*/data.parquet")
all_parquets = list(BASE.glob("*/*/*/data.parquet"))
# Sort by file size (descending)
single_file = max(all_parquets, key=lambda f: f.stat().st_size)

single_file

In [ ]:
# 1. Use Cartopy's Natural Earth admin_1_states_provinces shapefile
shp_path = shpreader.natural_earth(resolution='10m',
                                   category='cultural',
                                   name='admin_1_states_provinces')
states = gpd.read_file(shp_path)

# 2. Filter to California
ca = states[(states['admin'] == 'United States of America') &
            (states['name'] == 'California')]

# 3. Project to CA Albers for buffering
ca_proj = ca.to_crs("EPSG:3310")

# 4. Buffer by 20 km
buffered = ca_proj.buffer(1e5)

# 6. Convert to GeoSeries and back to WGS84
square_wgs = gpd.GeoSeries([buffered.geometry.iloc[0]], crs="EPSG:3310").to_crs("EPSG:4326")

In [ ]:
orig_geo_df = gpd.read_file(gpkg_path)
gdf = orig_geo_df.to_crs(epsg=4326)

degree_size = config.degree_size
gdf["lon_bin"] = (np.floor(gdf.geometry.x / degree_size) * degree_size).astype(float)
gdf["lat_bin"] = (np.floor(gdf.geometry.y / degree_size) * degree_size).astype(float)
gdf["grid_index"] = gdf.index

# ----------------------------------------------------------------------------
# 1) group by cell and collect both points AND their original indices
# ----------------------------------------------------------------------------
grouped = (
    gdf.groupby(["lon_bin", "lat_bin"])
    .agg(
        {
            "geometry": list,  # list of Point geometries
            "grid_index": list,  # list of original indices
        }
    )
    .reset_index()
)
grouped["cell_geom"] = grouped["geometry"].apply(make_cell_geom)  # type: ignore
cells = gpd.GeoDataFrame(grouped, geometry="cell_geom", crs="EPSG:4326")

In [ ]:
# Create the base map centered on the calculated location
centroid = square_wgs.geometry[0].centroid
base_map = folium.Map(location=[centroid.y, centroid.x], zoom_start=5, width=1000, height=800)

for idx, geo in enumerate(square_wgs):
    folium.GeoJson(
        geo,
        name=str(idx),
    ).add_to(base_map)

# Add each GeoJSON file to the map
# Add polygons to the map
intersects = gdf.geometry.intersects(square_wgs.geometry.iloc[0])
for idx, row in gdf[intersects].iterrows():
    pt = row["geometry"]
    folium.CircleMarker(
        location=[pt.y, pt.x],
        radius=0.5,
        color="red",
        fill=True,
        fill_opacity=0.1,
        popup=str(idx),
    ).add_to(base_map)

# Display the map
base_map

In [ ]:
# --- Connect to DuckDB ---
con = duckdb.connect()

In [ ]:
search_idx = gdf[gdf.geometry.intersects(square_wgs.geometry.iloc[0])].index[0]

for file in all_parquets:
    result = con.execute(
        f"SELECT COUNT(*) FROM read_parquet('{file}') WHERE grid_idx = {search_idx}"
    ).fetchone()[0]

    if result > 0:
        print(f"Found grid_idx {search_idx} in: {file}")
        single_file = file
        break

In [ ]:
# Register a view for all files
con.execute(f"""
    CREATE OR REPLACE VIEW samples_all AS
    SELECT * FROM '{all_files_pattern}'
""")

In [ ]:
# Register a view for a single file for faster iteration
con.execute(f"""
    CREATE OR REPLACE VIEW samples_one AS
    SELECT * FROM '{single_file}'
""")

In [ ]:
# --- Schema Inspection ---
print("Schema of samples_one:")
print(con.execute("DESCRIBE samples_one").fetchdf())

In [ ]:
# --- NULL Check ---
print("Checking for NULL values:")
df_nulls = con.execute("""
    SELECT
        SUM(CASE WHEN has_8_channel IS NULL THEN 1 ELSE 0 END) AS null_has_8_channel,
        SUM(CASE WHEN id IS NULL THEN 1 ELSE 0 END) AS null_id,
        SUM(CASE WHEN acquired IS NULL THEN 1 ELSE 0 END) AS null_acquired,
        SUM(CASE WHEN clear_percent IS NULL THEN 1 ELSE 0 END) AS null_clear_percent,
        SUM(CASE WHEN item_type IS NULL THEN 1 ELSE 0 END) AS null_item_type,
        SUM(CASE WHEN quality_category IS NULL THEN 1 ELSE 0 END) AS null_quality_category,
        SUM(CASE WHEN satellite_azimuth IS NULL THEN 1 ELSE 0 END) AS null_satellite_azimuth,
        SUM(CASE WHEN sun_azimuth IS NULL THEN 1 ELSE 0 END) AS null_sun_azimuth,
        SUM(CASE WHEN sun_elevation IS NULL THEN 1 ELSE 0 END) AS null_sun_elevation,
        SUM(CASE WHEN view_angle IS NULL THEN 1 ELSE 0 END) AS null_view_angle,
        SUM(CASE WHEN instrument IS NULL THEN 1 ELSE 0 END) AS null_instrument,
        SUM(CASE WHEN grid_idx IS NULL THEN 1 ELSE 0 END) AS null_grid_idx
    FROM samples_one
""").fetchdf()
print(df_nulls)

In [ ]:
# --- Preview Data ---
df_preview = con.execute("SELECT * FROM samples_one LIMIT 10").fetchdf()
display(df_preview)

In [ ]:
# --- Count Rows ---
print("Total rows in sample file:")
print(con.execute("SELECT COUNT(*) FROM samples_one").fetchone()[0])


In [ ]:
# --- Filter By Time ---
df_2024 = con.execute("""
    SELECT *
    FROM samples_one
    WHERE acquired >= '2024-01-01' AND acquired < '2025-01-01'
    LIMIT 100
""").fetchdf()
display(df_2024)

In [ ]:
# Determine which cell this file corresponds to by matching its grid_idx
file_grid_idxs = con.execute("SELECT DISTINCT grid_idx FROM samples_one").fetchdf()["grid_idx"].tolist()
cells_containing_file_data = cells[cells["grid_index"].apply(lambda lst: any(idx in file_grid_idxs for idx in lst))]

# Flatten all grid indices in the matched cell
matching_indices = set(idx for lst in cells_containing_file_data["grid_index"] for idx in lst)
geo_filtered = gdf[gdf["grid_index"].isin(matching_indices)]

In [ ]:
# --- Load Geo Points and Join ---

# Sample count per grid cell
df_counts = con.execute("""
    SELECT grid_idx, COUNT(*) as sample_count
    FROM samples_one
    GROUP BY grid_idx
""").fetchdf()

geo_plot = geo_filtered.join(df_counts.set_index("grid_idx"), how="left").fillna({"sample_count": 0})

# --- Create Folium Map of Sample Count ---
if geo_plot.sample_count.max() == geo_plot.sample_count.min():
    scale_min = 0
else:
    scale_min = geo_plot.sample_count.min()
color_scale = linear.viridis.scale(scale_min, geo_plot.sample_count.max())
m = folium.Map(
    location=[geo_plot.geometry.y.mean(), geo_plot.geometry.x.mean()], 
    zoom_start=8, 
    tiles="CartoDB positron",
    width=500, 
    height=500
)

for _, row in geo_plot.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=color_scale(row.sample_count),
        popup=f"Grid ID: {row.name}<br>Samples: {row.sample_count}"
    ).add_to(m)

color_scale.caption = "Sample Count"
color_scale.width = 300  # Wider legend
color_scale.heght = 100  # Wider legend
color_scale.add_to(m)

m

In [ ]:
# --- Histogram of Sample Count per Month ---
df_monthly = con.execute("""
    SELECT date_trunc('month', acquired) AS month, COUNT(DISTINCT id) AS sample_count
    FROM samples_one
    WHERE item_type = 'PSScene'
    GROUP BY month
    ORDER BY month
""").fetchdf()

# Plot histogram
plt.figure(figsize=(10,4))
plt.bar(df_monthly['month'].astype(str), df_monthly['sample_count'])
plt.xticks(rotation=45, ha='right')
plt.title("Sample Count per Month (Unique Scenes)")
plt.xlabel("Month")
plt.ylabel("Sample Count")
plt.tight_layout()
plt.show()

In [ ]:
# --- Fraction of has_8_channel per Grid Point ---
df_fraction_8ch = con.execute("""
    SELECT grid_idx,
           COUNT(*) AS total,
           SUM(CASE WHEN has_8_channel THEN 1 ELSE 0 END)::DOUBLE / COUNT(*) AS frac_8_channel
    FROM samples_one
    WHERE item_type = 'PSScene'
    GROUP BY grid_idx
""").fetchdf()

geo_frac = geo_filtered.join(df_fraction_8ch.set_index("grid_idx"), how="left").fillna({"frac_8_channel": 0})

# --- Folium map for 8-channel fraction ---
color_scale_frac = linear.viridis.scale(0, 1)
m2 = folium.Map(
    location=[geo_frac.geometry.y.mean(), geo_frac.geometry.x.mean()], 
    zoom_start=8, 
    tiles="CartoDB positron",
    width=500,
    height=500
)

for _, row in geo_frac.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=3,
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=color_scale_frac(row.frac_8_channel),
        popup=f"Grid ID: {row.name}<br>8-Chan Fraction: {row.frac_8_channel:.2f}"
    ).add_to(m2)

color_scale_frac.caption = "Fraction of 8-Channel Observations"
color_scale_frac.width = 300  # Wider legend
color_scale_frac.add_to(m2)

m2

In [ ]:
unique_instruments = con.execute("SELECT DISTINCT instrument FROM samples_one").fetchdf()
print(unique_instruments)

In [ ]:
# --- Fraction of has_8_channel per Grid Point ---
df_fraction_8ch_all = con.execute("""
    SELECT grid_idx,
           COUNT(*) AS total,
           SUM(CASE WHEN has_8_channel THEN 1 ELSE 0 END)::DOUBLE / COUNT(*) AS frac_8_channel
    FROM samples_all
    WHERE item_type = 'PSScene'
    GROUP BY grid_idx
""").fetchdf()

geo_frac = gdf.join(df_fraction_8ch_all.set_index("grid_idx"), how="inner")

# Set up the figure
fig, ax = plt.subplots(figsize=(14, 7))

# Optional: Plot world basemap
# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# world.plot(ax=ax, color='lightgray', edgecolor='white')

# Plot points with frac_8_channel shading
geo_frac.plot(
    ax=ax,
    column="frac_8_channel",
    cmap="plasma",
    markersize=10,
    legend=True,
    vmin=0.5,  # Force lower bound
    vmax=1.0,  # Force upper bound
    legend_kwds={
        "label": "Fraction of 8-Channel Observations",
        "shrink": 0.7,
        "orientation": "vertical"
    }
)

ax.set_title("Fraction of 8-Channel Observations per Grid Point", fontsize=14)
ax.set_axis_off()
plt.tight_layout()
plt.show()